In [1]:
import os
os.chdir('..')

In [2]:
import inspect
import multiprocessing as mp
from typing import List, Tuple

import numba as nb
import numpy as np
import pandas as pd
import time

from neo_backtesting import Backtester
from strategy import boll_inbar
from util import read_candle_feather, transform_candle_np_struct, transform_np_struct
from joblib import Parallel, delayed

%load_ext line_profiler

In [3]:
# 回测起始日
START_DATE = '20180301'
END_DATE = '20220614'

# 布林带周期与宽度
N = 100
B = 2

# 回测参数
INIT_CAPITAL = 1e5  # 初始资金，10万
FACE_VALUE = 0.001  # 合约面值 0.001

COMM_RATE = 6e-4  # 交易成本万分之 6
LIQUI_RATE = 5e-3  # 爆仓保证金率千分之 5

CONTRACT_TYPE = 'futures'  # 正向合约

# 模拟器参数
SIMULATOR_PARAMS = {
    'init_capital': INIT_CAPITAL, 
    'face_value': FACE_VALUE, 
    'comm_rate': COMM_RATE, 
    'liqui_rate': LIQUI_RATE, 
    'init_pos': 0
}

STRA = boll_inbar  # 要回测的策略

MIN_INTERVAL = '1m'  # 最小回测周期

ETH_PATHS = {
    '1m': '/mnt/data2/crypto_data/feather_data/spot/ETH-USDT_1m.fea',
    '30m': '/mnt/data2/crypto_data/feather_data/spot/ETH-USDT_30m.fea',
    '1h': '/mnt/data2/crypto_data/feather_data/spot/ETH-USDT_1h.fea'
}

In [4]:
%%time
backtester = Backtester(
    candle_paths=ETH_PATHS, 
    contract_type='futures', 
    simulator_params=SIMULATOR_PARAMS, 
    stra_module=boll_inbar)
factor_params = {
    'n': 100,
    'b': 2,
    'itl': '1h'
}
strategy_params = {
    'leverage': 1
}
results = backtester.run_detailed(
    start_date=START_DATE, 
    end_date=END_DATE, 
    init_capital=INIT_CAPITAL, 
    face_value=FACE_VALUE, 
    factor_params=factor_params, 
    strategy_params=strategy_params)

results['equity'] /= results['equity'].iat[0]
results

CPU times: user 2.71 s, sys: 2.5 s, total: 5.21 s
Wall time: 3.15 s


,candle_begin_time,candle_end_time,open,high,low,close,volume,upper,median,lower,pos,equity
275636,2018-03-01 00:00:00,2018-03-01 00:01:00,853.50,853.75,852.75,853.00,13.72510,898.871249,858.6274,818.383551,0,1.000000
275637,2018-03-01 00:01:00,2018-03-01 00:02:00,853.51,853.51,852.60,852.80,20.10843,898.871249,858.6274,818.383551,0,1.000000
275638,2018-03-01 00:02:00,2018-03-01 00:03:00,853.41,853.41,852.80,853.01,26.58684,898.871249,858.6274,818.383551,0,1.000000
275639,2018-03-01 00:03:00,2018-03-01 00:04:00,853.01,853.39,852.61,852.97,19.51853,898.871249,858.6274,818.383551,0,1.000000
275640,2018-03-01 00:04:00,2018-03-01 00:05:00,852.97,852.97,850.99,851.00,100.14852,898.871249,858.6274,818.383551,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2524870,2022-06-13 23:56:00,2022-06-13 23:57:00,1206.46,1209.86,1205.19,1207.06,727.10310,1903.122423,1534.1303,1165.138177,-5165344,115.957543
2524871,2022-06-13 23:57:00,2022-06-13 23:58:00,1207.07,1209.94,1205.86,1205.93,774.45520,1903.122423,1534.1303,1165.138177,-5165344,116.015911
2524872,2022-06-13 23:58:00,2022-06-13 23:59:00,1205.93,1207.62,1203.87,1206.07,550.73550,1903.122423,1534.1303,1165.138177,-5165344,116.008680
2524873,2022-06-13 23:59:00,2022-06-14 00:00:00,1206.07,1212.76,1205.03,1209.82,1152.11320,1899.522235,1528.3911,1157.259965,-5165344,115.814979


In [5]:
%%time

fparams_list = STRA.get_default_factor_params_list()
sparams_list = STRA.get_default_strategy_params_list()

backtester = Backtester(
    candle_paths=ETH_PATHS, 
    contract_type='futures', 
    simulator_params=SIMULATOR_PARAMS, 
    stra_module=boll_inbar)
df = backtester.run_gridsearch(
    start_date=START_DATE,
    end_date=END_DATE,
    init_capital=INIT_CAPITAL,
    face_value=FACE_VALUE,
    fparams_list=fparams_list,
    sparams_list=sparams_list
)

CPU times: user 1min 43s, sys: 1min 3s, total: 2min 47s
Wall time: 2min 46s


In [6]:

fparams_list = STRA.get_default_factor_params_list()
sparams_list = STRA.get_default_strategy_params_list()

fparams_seqs = []

n = len(fparams_list)
n_cpus = os.cpu_count() // 2

j = 0
for i in range(n_cpus):
    n_tasks = n // n_cpus
    if i < n % n_cpus:
        n_tasks += 1
    fparams_seqs.append(fparams_list[j : j + n_tasks])
    j += n_tasks

def search(fl):
    backtester = Backtester(
        candle_paths=ETH_PATHS, 
        contract_type='futures', 
        simulator_params=SIMULATOR_PARAMS, 
        stra_module=boll_inbar)
    t1 = time.time()
    df = backtester.run_gridsearch(
        start_date=START_DATE,
        end_date=END_DATE,
        init_capital=INIT_CAPITAL,
        face_value=FACE_VALUE,
        fparams_list=fl,
        sparams_list=sparams_list
    )
    return time.time() - t1, len(fl)

%time times = Parallel(n_jobs=n_cpus)(delayed(search)(fl) for fl in fparams_seqs)

CPU times: user 24.1 ms, sys: 167 ms, total: 191 ms
Wall time: 48.7 s


In [8]:
%%time 
with mp.Pool() as pool:
    pool.map(search, fparams_seqs)

CPU times: user 20.6 ms, sys: 336 ms, total: 356 ms
Wall time: 46.6 s
